<a href="https://colab.research.google.com/github/youngpaper1115/Git_Tutorial/blob/main/Lending_Club_Project_ver2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0. 기본 파트

In [ ]:
#google drive mount 및 파일 경로 지정

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 필요 라이브러리 import 및 현재 작업공간 확인

# 기본 라이브러리
import pandas as pd
import numpy as np
import os

# 시각화 라이브러리
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib notebook
%matplotlib inline

# 통계 및 기계학습 관련 라이브러리
from scipy import stats    # 통계 관련 라이브러리
from sklearn.model_selection import train_test_split, RandomizedSearchCV    # 데이터 셋 분류 라이브러리
from sklearn.utils import resample    # 데이터 업샘플링&다운샘플링 라이브러리
from sklearn.preprocessing import RobustScaler    # 데이터 스케일링 라이브러리
from sklearn.metrics import (accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve, auc, plot_confusion_matrix, plot_roc_curve)    # 모델 정확도 검증 라이브러리
from sklearn.linear_model import LogisticRegression    # 로지스틱 회귀 라이브러리
from xgboost import XGBClassifier    # XGBoost 라이브러리
from sklearn.ensemble import RandomForestClassifier    # 랜덤포레스트 라이브러리

# 작업공간 확인
os.getcwd()

In [ ]:
# 데이터 불러오기
lc_df = pd.read_stata('/content/drive/MyDrive/Python_SNU/Lending_Club/lendingclub.dta')

# emp_length 결과 불러오기
result_list = []
f = open('/content/drive/MyDrive/Python_SNU/Lending_Club/emp_length.txt', 'r')
numb = f.read()
f.close()
result_list = numb.split(",")
del result_list[-1]

# raw data에 emp_length 컬럼 추가
lc_df['emp_length'] = result_list
lc_df = lc_df.astype({'emp_length': 'int'})

## 1-1. 이상치 제거
- 이상치가 존재하지 않는 더미 변수 or 범주 변수는 따로 분석하지 않음
- 상자 그림(Box Plot)을 참고했을 때, 1.5 IQR(Inter Quantile Range)를 기준으로 이상치를 제거할 경우, 많은 데이터 양이 손실될 것으로 보임 => 다른 기준 고민

(1) annual_inc : 너무 큰 값이 1개 존재하여 이를 제거

In [ ]:
lc_df.boxplot(column = ["annual_inc"], by = ["depvar"])
plt.show()

In [ ]:
lc_df = lc_df.drop(index = lc_df[lc_df['annual_inc'] == lc_df['annual_inc'].max()].index)    # annual_inc 변수의 max값 index를 찾은 후 해당 데이터 제거
lc_df.boxplot(column = ["annual_inc"], by = ["depvar"])
plt.show()

(2) installment

In [ ]:
lc_df.boxplot(column = ["installment"], by = ["depvar"])
plt.show()

(3) dti

In [ ]:
lc_df.boxplot(column = ["dti"], by = ["depvar"])
plt.show()

(4) open_acc, total_acc

In [ ]:
lc_df.boxplot(column = ["open_acc", "total_acc"], by = ["depvar"])
plt.show()

## 1-2. 변수 추가
- home_ownership : 1,2 / 3, 4, 5, 6 으로 분류 => home_ownership12 컬럼 추가(3 or 4 or 5 or 6인 경우 값이 1)
- mths_since_rcnt_il : 1 / 2 ~ 5 / 6 ~ 11로 구분 => 각각 mths_since_rcnt_il_a, mths_since_rcnt_il_b, mths_since_rcnt_il_c 로 생성
- mths_since_recent_bc : 1 ~ 6 / 7 ~ 11로 구분 => mths_since_recent_bc 열 생성(1 ~ 6인 경우 값이 1)
- mths_since_recent_inq : 1 / 2 ~ 6 / 7 ~ 11 로 구분 => 각각 mths_since_recent_inq_a, mths_since_recent_inq_b, mths_since_recent_inq_c 로 생성
- delinq_2yrs : 0 / 1 ~ 11 / 12 ~ 나머지 => 각각 delinq_2yrs_a, delinq_2yrs_b, delinq_2yrs_c 로 생성
- fico_range_low : 5구간으로 나눔 => fico_range_low_a 생성
- inq_last_6mths : 0 / 1 / 2~ 로 구분 => 각각 inq_last_6mths_a, inq_last_6mths_b, inq_last_6mths_c 로 생성
- collections_12_mths_ex_med : 0 / 나머지 => collections_12_mths_ex_med_a 열 생성 (0값은 그대로 0)
- acc_now_delinq : 0 / 나머지로 우선 => acc_now_delinq_a 열 생성 (0 값은 그대로 0)
- pub_rec_bankruptcies : 0과 나머지 => pub_rec_bankruptcies_a 열 생성 (0 값은 그대로 0)
- tax_liens : 0과 나머지 => tax_liens_a 열 생성 (0 값은 그대로 0)
- installment : 5구간으로 나눔 => installment_a 열 생성
- annual_inc : 5구간으로 나눔 => annual_inc_a
- collection_recovery_fee : 3구간으로 나눔 => collection_recovery_fee_a
- dti : 5구간으로 나눔 => dti_a

In [ ]:
# home_ownership 부분
lc_df['home_ownership12'] = 0
change_index = lc_df[(lc_df['home_ownership3'] == 1) | (lc_df['home_ownership4'] == 1) | (lc_df['home_ownership5'] == 1) | (lc_df['home_ownership6'] == 1)].index.tolist()
lc_df.loc[change_index, 'home_ownership12'] = 1

# mths_since_rcnt_il 부분
lc_df['mths_since_rcnt_il_a'] = lc_df['mths_since_rcnt_il1']
lc_df['mths_since_rcnt_il_b'] = 0
change_index = lc_df[(lc_df['mths_since_rcnt_il2'] == 1) | (lc_df['mths_since_rcnt_il3'] == 1) | (lc_df['mths_since_rcnt_il4'] == 1) | (lc_df['mths_since_rcnt_il5'] == 1)].index.tolist()
lc_df.loc[change_index, 'mths_since_rcnt_il_b'] = 1
lc_df['mths_since_rcnt_il_c'] = 0
change_index = lc_df[(lc_df['mths_since_rcnt_il6'] == 1) | (lc_df['mths_since_rcnt_il7'] == 1) | (lc_df['mths_since_rcnt_il8'] == 1) | (lc_df['mths_since_rcnt_il9'] == 1) | (lc_df['mths_since_rcnt_il10'] == 1)| (lc_df['mths_since_rcnt_il11'] == 1)].index.tolist()
lc_df.loc[change_index, 'mths_since_rcnt_il_c'] = 1

# mths_since_recent_bc 부분
lc_df['mths_since_recent_bc'] = 0
change_index = lc_df[(lc_df['mths_since_recent_bc1'] == 1) | (lc_df['mths_since_recent_bc2'] == 1) | (lc_df['mths_since_recent_bc3'] == 1) | (lc_df['mths_since_recent_bc4'] == 1) | (lc_df['mths_since_recent_bc5'] == 1) | (lc_df['mths_since_recent_bc6'] == 1)].index.tolist()
lc_df.loc[change_index, 'mths_since_recent_bc'] = 1

# mths_since_recent_inq 부분
lc_df['mths_since_recent_inq_a'] = lc_df['mths_since_recent_inq1']
lc_df['mths_since_recent_inq_b'] = 0
change_index = lc_df[(lc_df['mths_since_recent_inq2'] == 1) | (lc_df['mths_since_recent_inq3'] == 1) | (lc_df['mths_since_recent_inq4'] == 1) | (lc_df['mths_since_recent_inq5'] == 1) | (lc_df['mths_since_recent_inq6'] == 1)].index.tolist()
lc_df.loc[change_index, 'mths_since_recent_inq_b'] = 1
lc_df['mths_since_recent_inq_c'] = 0
change_index = lc_df[(lc_df['mths_since_recent_inq7'] == 1) | (lc_df['mths_since_recent_inq8'] == 1) | (lc_df['mths_since_recent_inq9'] == 1) | (lc_df['mths_since_recent_inq10'] == 1)].index.tolist()
lc_df.loc[change_index, 'mths_since_recent_inq_c'] = 1

# delinq_2yrs 부분
lc_df['delinq_2yrs_a'] = 0
change_index = lc_df[lc_df['delinq_2yrs'] == 0].index.tolist()
lc_df.loc[change_index, 'delinq_2yrs_a'] = 1
lc_df['delinq_2yrs_b'] = 0
change_index = lc_df[(lc_df['delinq_2yrs'] >= 1) & (lc_df['delinq_2yrs'] <= 11)].index.tolist()
lc_df.loc[change_index, 'delinq_2yrs_b'] = 1
lc_df['delinq_2yrs_c'] = 0
change_index = lc_df[lc_df['delinq_2yrs'] >= 12].index.tolist()
lc_df.loc[change_index, 'delinq_2yrs_c'] = 1

# fico_range_low 부분
lc_df['fico_range_low_a'] = pd.cut(lc_df['fico_range_low'], 5, labels = list(range(5)))

# inq_last_6mths 부분
lc_df['inq_last_6mths_a'] = 0
change_index = lc_df[lc_df['inq_last_6mths'] == 0].index.tolist()
lc_df.loc[change_index, 'inq_last_6mths_a'] = 1
lc_df['inq_last_6mths_b'] = 0
change_index = lc_df[lc_df['inq_last_6mths'] == 1].index.tolist()
lc_df.loc[change_index, 'inq_last_6mths_b'] = 1
lc_df['inq_last_6mths_c'] = 0
change_index = lc_df[lc_df['inq_last_6mths'] >= 2].index.tolist()
lc_df.loc[change_index, 'inq_last_6mths_c'] = 1

# collections_12_mths_ex_med 부분
lc_df['collections_12_mths_ex_med_a'] = 1
change_index = lc_df[lc_df['collections_12_mths_ex_med'] == 0].index.tolist()
lc_df.loc[change_index, 'collections_12_mths_ex_med_a'] = 0

# acc_now_delinq 부분
lc_df['acc_now_delinq_a'] = 1
change_index = lc_df[lc_df['acc_now_delinq'] == 0].index.tolist()
lc_df.loc[change_index, 'acc_now_delinq_a'] = 0

# pub_rec_bankruptcies 부분
lc_df['pub_rec_bankruptcies_a'] = 1
change_index = lc_df[lc_df['pub_rec_bankruptcies'] == 0].index.tolist()
lc_df.loc[change_index, 'pub_rec_bankruptcies_a'] = 0

# tax_liens 부분
lc_df['tax_liens_a'] = 1
change_index = lc_df[lc_df['tax_liens'] == 0].index.tolist()
lc_df.loc[change_index, 'tax_liens_a'] = 0

# installment 부분
lc_df['installment_a'] = pd.cut(lc_df['installment'], 5, labels = list(range(5)))

# annual_inc 부분
lc_df['annual_inc_a'] = pd.qcut(lc_df['annual_inc'], q = 5, labels = [1, 2, 3, 4, 5])

# collection_recovery_fee 부분
lc_df['collection_recovery_fee_a'] = pd.cut(lc_df['collection_recovery_fee'], 3, labels = list(range(3)))

# dti 부분
lc_df['dti_a'] = pd.qcut(lc_df['dti'], q = 5, labels = [1, 2, 3, 4, 5])

## 2. 데이터 분석 진행 : 순서는 아래와 같음
(1) 변수 추출 및 데이터 생성

(2) Train / Test Set 구분 (필요시 Scaling 진행)

(3) Logistic / Random Forest / XGBoost 모델 생성 및 정확도 평가 (필요시 다운샘플링 데이터 Set도 적용)





In [ ]:
# 모델 정확도 확인 함수 정의

def print_score(true, pred, train = True):
    if train:
        clf_report = pd.DataFrame(classification_report(true, pred, output_dict = True))
        print("Train Result:\n================================================")
        print(f"Accuracy Score: {accuracy_score(true, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(true, pred)}\n")
        
    elif train==False:
        clf_report = pd.DataFrame(classification_report(true, pred, output_dict = True))
        print("Test Result:\n================================================")        
        print(f"Accuracy Score: {accuracy_score(true, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(true, pred)}\n")

### 2-1. 5조가 선택한 변수로 분석 진행

- 변수 추출

In [ ]:
data = lc_df[['emp_length12', 'home_ownership12', 'mths_since_last_delinq1', 'mths_since_last_major_derog1', 'mths_since_last_record1', 'mths_since_rcnt_il_a', 'mths_since_rcnt_il_b', 'mths_since_rcnt_il_c', 
              'mths_since_recent_bc', 'mths_since_recent_bc_dlq1', 'mths_since_recent_inq_a', 'mths_since_recent_inq_b', 'mths_since_recent_inq_c', 'delinq_2yrs_a', 'delinq_2yrs_b', 'delinq_2yrs_c', 
              'fico_range_low_a', 'inq_last_6mths_a', 'inq_last_6mths_b', 'inq_last_6mths_c', 'collections_12_mths_ex_med_a', 'acc_now_delinq_a', 'pub_rec_bankruptcies_a', 'tax_liens_a', 'installment_a', 
              'annual_inc_a', 'collection_recovery_fee_a', 'dti_a', 'depvar']]

In [ ]:
# 종속변수(depvar)의 값 개수 확인

w_good = data.depvar.value_counts()[0] / data.shape[0]
w_bad = data.depvar.value_counts()[1] / data.shape[0]

print(f"우량 고객(Good) 비율 : %.3f"%w_good)
print(f"불량 고객(Bad) 비율 : %.3f"%w_bad)

data['depvar'].value_counts().plot.bar(title = "Loan Status Counts", xlabel = 'Loan Status', ylabel = 'Count')

In [ ]:
feature_columns = list(data.columns.difference(['depvar']))
X = data[feature_columns]    # 변인변수들의 데이터 Set
y = data['depvar']    # 종속변수 데이터

- Train Set / Test Set으로 분류 (비율은 8:2)

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.2, random_state = 42)

print(train_X.shape, test_X.shape, train_y.shape, test_y.shape)

(1) 로지스틱 회귀분석 진행 (변인변수가 모두 더미 변수이거나 범주형 변수여서 따로 Scaling은 진행하지 않음) : 약 15초 소요

In [ ]:
# 로지스틱 회귀분석 모델 생성
lr_clf = LogisticRegression() 

# 모델 학습 진행
lr_clf.fit(train_X, np.ravel(train_y))

- 모델 정확도 확인 : Train Set과 Test Set의 정확도 모두 83.8%의 정확도를 보였다.
우량 고객 비율 83.8%와 동일한 값으로, 이는 모델이 어떤 값이 입력되든 0의 결과를 내뱉는 모델로 학습된 것을 의미한다. 즉, 제대로 학습된 모델로 볼 수 없다.

In [ ]:
# 모델 정확도 확인

train_y_pred = lr_clf.predict(train_X)
test_y_pred = lr_clf.predict(test_X)

print_score(train_y, train_y_pred, train = True)
print_score(test_y, test_y_pred, train = False)

- Confusion Matrix와 AUC-ROC Curve 확인 : Confusion Matrix를 봤을 때 Bad로 예측한 경우는 극히 일부밖에 없는 것을 확인할 수 있다. AUC 값 역시 0.64로 해당 분류 모델은 우량/불량 클래스를 구분할 수 있는 능력이 있다고 보기 힘들다.

In [ ]:
# confusion matrix, AUC-ROC 커브 생성

disp = plot_confusion_matrix(lr_clf, test_X, test_y, cmap='Blues', values_format='d', display_labels=['Good', 'Bad'])
disp = plot_roc_curve(lr_clf, test_X, test_y)

(2) Random Forest 진행 : 약 2분 소요

In [ ]:
# Random Forest 모델 생성
rf_clf = RandomForestClassifier(n_estimators = 50)    # n_estimators는 생성할 Tree의 개수. 개수가 많을수록 정확도가 올라가지만 모델 학습 시간이 길어짐

# 모델 학습 진행
rf_clf.fit(train_X, train_y)

- 모델 정확도 확인 : Test Set의 정확도는 86.3%로 로지스틱 회귀 모델보다 살짝 좋아진 것을 확인할 수 있다. 하지만 Test Set의 결과는 82.3%로 살짝 하락한 상태이다.

In [ ]:
# 모델 정확도 확인

train_y_pred = rf_clf.predict(train_X)
test_y_pred = rf_clf.predict(test_X)

print_score(train_y, train_y_pred, train = True)
print_score(test_y, test_y_pred, train = False)

- Confusion Matrix와 AUC-ROC Curve 확인 : AUC 값이 0.59로 모델의 정확도가 매우 떨어짐을 확인할 수 있다.

In [ ]:
# confusion matrix, AUC-ROC 커브 생성

disp = plot_confusion_matrix(rf_clf, test_X, test_y, cmap='Blues', values_format='d', display_labels=['Good', 'Bad'])
disp = plot_roc_curve(rf_clf, test_X, test_y)

(3) XGBoost 진행

In [ ]:
# XGBoost 모델 생성
xgb_clf = XGBClassifier(use_label_encoder = False)    # XGBoost 라이브러리 문서까지 찾아봤는데, 별다른 설명 없이 False를 추천한다 하여 False로 설정

# 모델 학습 진행
xgb_clf.fit(train_X, train_y)

- 모델 정확도 확인 : Train Set, Test Set 모두 로지스틱 회귀 모델과 비슷한 정확도를 보였기에 좋다고 볼 수 없다.

In [ ]:
# 모델 정확도 확인

train_y_pred = xgb_clf.predict(train_X)
test_y_pred = xgb_clf.predict(test_X)

print_score(train_y, train_y_pred, train = True)
print_score(test_y, test_y_pred, train = False)

- Confusion Matrix와 AUC-ROC Curve 확인 : Bad로 판단한 경우가 매우 적다. 또한 AUC 값도 0.65로 높다고 보기 힘들다.

In [ ]:
# confusion matrix, AUC-ROC 커브 생성

disp = plot_confusion_matrix(xgb_clf, test_X, test_y, cmap='Blues', values_format='d', display_labels=['Good', 'Bad'])
disp = plot_roc_curve(xgb_clf, test_X, test_y)

### 2-2. 2-1의 추출 데이터 기반, 다운샘플링 진행

- Data Set의 다운샘플링 진행

In [ ]:
# 다운샘플링 이전 데이터셋 비율 확인 (Good : 83.8%, Bad : 16.2%)

print(data['depvar'].value_counts())
data.groupby('depvar').size().plot(kind='pie', y = 'depvar', label = "Type", autopct='%1.1f%%')

In [ ]:
# 우량 / 불량에 따른 데이터 분리
good_data = data[data['depvar'] == 0]
bad_data  = data[data['depvar'] == 1]

# 우량 데이터 수가 많기 때문에 우량 데이터에 대한 다운샘플링 진행
good_downsample = resample(good_data, replace = True, n_samples = len(bad_data), random_state=42)

# 다운샘플링한 우량 데이터와 기존 불량 데이터를 재결합한 Data Set 생성
data_dsp = pd.concat([good_downsample, bad_data])

# 확인 (Good : 50%, Bad : 50%)
data_dsp.groupby('depvar').size().plot(kind='pie', y = 'depvar', label = "Type", autopct='%1.1f%%')

- Train / Test Set 재설정 (비율은 이전과 동일하게 8:2)

In [ ]:
feature_columns = list(data_dsp.columns.difference(['depvar']))
X = data_dsp[feature_columns]    # 변인변수들의 데이터 Set
y = data_dsp['depvar']    # 종속변수 데이터를

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.2, random_state = 42)

print(train_X.shape, test_X.shape, train_y.shape, test_y.shape)

In [ ]:
train_X = np.array(train_X).astype(np.float32)
test_X = np.array(test_X).astype(np.float32)
train_y = np.array(train_y).astype(np.float32)
test_y = np.array(test_y).astype(np.float32)

(1) 로지스틱 회귀분석 진행

In [ ]:
# 로지스틱 회귀분석 모델 생성
lr_clf = LogisticRegression() 

# 모델 학습 진행
lr_clf.fit(train_X, np.ravel(train_y))

- 모델 정확도 확인 : 60%의 정확도를 보이며 다운샘플링 이전 데이터로 학습한 모델보다는 변별력이 있는 것으로 보이지만, 여전히 그 수치가 낮음. F1 Score 역시 0.59를 기록하며 정확도가 낮은 것을 확인할 수 있음

In [ ]:
# 모델 정확도 확인

train_y_pred = lr_clf.predict(train_X)
test_y_pred = lr_clf.predict(test_X)

print_score(train_y, train_y_pred, train = True)
print_score(test_y, test_y_pred, train = False)

- Confusion Matrix와 AUC-ROC Curve 확인 : AUC 값이 0.65로 역시 낮다.

In [ ]:
# confusion matrix, AUC-ROC 커브 생성

disp = plot_confusion_matrix(lr_clf, test_X, test_y, cmap='Blues', values_format='d', display_labels=['Good', 'Bad'])
disp = plot_roc_curve(lr_clf, test_X, test_y)

(2) Random Forest 진행 : 약 17초 소요(Data 수 감소 영향인 듯)

In [ ]:
# Random Forest 모델 생성
rf_clf = RandomForestClassifier(n_estimators = 50)    # n_estimators는 생성할 Tree의 개수. 개수가 많을수록 정확도가 올라가지만 모델 학습 시간이 길어짐

# 모델 학습 진행
rf_clf.fit(train_X, train_y)

- 모델 정확도 확인 : Train Set의 예측은 75.47%로 향상되었지만, Test Set의 예측은 57.69%로 차이가 크게 난다. F1 Score도 75%에서 57%로 크게 떨어졌다.

In [ ]:
# 모델 정확도 확인

train_y_pred = rf_clf.predict(train_X)
test_y_pred = rf_clf.predict(test_X)

print_score(train_y, train_y_pred, train = True)
print_score(test_y, test_y_pred, train = False)

- Confusion Matrix와 AUC-ROC Curve 확인 : AUC 값 역시 낮다

In [ ]:
# confusion matrix, AUC-ROC 커브 생성

disp = plot_confusion_matrix(rf_clf, test_X, test_y, cmap='Blues', values_format='d', display_labels=['Good', 'Bad'])
disp = plot_roc_curve(rf_clf, test_X, test_y)

(3) XGBoost 진행

In [ ]:
# XGBoost 모델 생성
xgb_clf = XGBClassifier(use_label_encoder = False)    # XGBoost 라이브러리 문서까지 찾아봤는데, 별다른 설명 없이 False를 추천한다 하여 False로 설정

# 모델 학습 진행
xgb_clf.fit(train_X, train_y)

- 모델 정확도 확인 : 60%대. F1 score도 비슷

In [ ]:
# 모델 정확도 확인

train_y_pred = xgb_clf.predict(train_X)
test_y_pred = xgb_clf.predict(test_X)

print_score(train_y, train_y_pred, train = True)
print_score(test_y, test_y_pred, train = False)

- Confusion Matrix와 AUC-ROC Curve 확인 : AUC 값 역시 마찬가지...

In [ ]:
# confusion matrix, AUC-ROC 커브 생성

disp = plot_confusion_matrix(xgb_clf, test_X, test_y, cmap='Blues', values_format='d', display_labels=['Good', 'Bad'])
disp = plot_roc_curve(xgb_clf, test_X, test_y)

### 2-3. 많은 변수들을 넣고 돌려보기

- 우선은 많은 변수들을 넣고 모델을 생성해보지만, 이 경우는 당연히 정확도는 높게 나올 것(과적합이 될 것으로 예상되기 때문에)

- 우리는 과적합을 최대한 낮추면서 정확도도 높일 수 있는 방안을 찾는 것이 중요 -> 이를 위해서는 가장 설명력이 높은 변수들을 몇개만 고르는 것이 바람직해보임

- 우선 제거한 변수들 목록

    - 필요x 제거 : purpose, issue_d, elapsed_t, addr_state, mths_since_recent_revol_delinq
    - 범주화로 제거 : emp_length 1 ~ 12, home_ownership 1 ~ 6, mths_since_last_delinq 2 ~ 11, mths_since_last_major_derog 2 ~ 11, mths_since_last_record 2 ~ 11, mths_since_rcnt_il 1 ~ 11, mths_since_recent_bc 1 ~ 11, mths_since_recent_bc_dlq 2 ~ 11, mths_since_recent_inq 1  ~ 10
    - 관련 변수 제거 : installment(loan_amnt, funded_amnt, funded_amnt_inv, total_pymnt, total_pymnt_inv), recoveries(collection_recovery_fee), fico_range_low(fico_range_high), last_fico_range_low(last_fico_range_high), out_prncp(out_prncp_inv), initial_list_status1(initial_list_status2), mths_since_recent_inq_b(mths_since_recent_inq_c), total_rec_int(total_rec_prncp)
    - 원래 변수 1개로 처리되던 것들은 따로 범주화 진행 X

- 변수 추출

In [ ]:
data = lc_df[['int_rate', 'installment', 'annual_inc', 'dti', 'delinq_2yrs', 'fico_range_low', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 
              'total_acc', 'out_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'last_pymnt_amnt', 'last_fico_range_low', 
              'collections_12_mths_ex_med', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'chargeoff_within_12_mths', 'delinq_amnt', 'pub_rec_bankruptcies', 'tax_liens', 
              'verification_status1', 'verification_status2', 'verification_status3', 'initial_list_status1', 'debt_settlement_flag1', 'term1', 'depvar', 
              'mths_since_last_delinq1', 'mths_since_last_major_derog1', 'mths_since_last_record1', 'mths_since_recent_bc_dlq1', 'emp_length', 'home_ownership12', 
              'mths_since_rcnt_il_a', 'mths_since_rcnt_il_b', 'mths_since_rcnt_il_c', 'mths_since_recent_bc', 'mths_since_recent_inq_a', 'mths_since_recent_inq_b']]

In [ ]:
# 종속변수(depvar)의 값 개수 확인

w_good = data.depvar.value_counts()[0] / data.shape[0]
w_bad = data.depvar.value_counts()[1] / data.shape[0]

print(f"우량 고객(Good) 비율 : %.3f"%w_good)
print(f"불량 고객(Bad) 비율 : %.3f"%w_bad)

data['depvar'].value_counts().plot.bar(title = "Loan Status Counts", xlabel = 'Loan Status', ylabel = 'Count')

In [ ]:
feature_columns = list(data.columns.difference(['depvar']))
X = data[feature_columns]    # 변인변수들의 데이터 Set
y = data['depvar']    # 종속변수 데이터

- Train Set / Test Set으로 분류 (비율은 8:2)

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.2, random_state = 42)

print(train_X.shape, test_X.shape, train_y.shape, test_y.shape)

(1) 로지스틱 회귀분석 진행 : 위에서 진행한 내용과 동일하므로 한 셀에서 진행.. (약 40초 소요)

- Train / Test Set 예측 정확도가 96.5%로 나쁘지 않음. F1-score 역시 97.9
- AUC 역시 0.98로 높은 값을 보여줌.
- 다만 너무 높은 정확도로, 역시 과적합(overfitting)을 의심해보지 않을 수 없음

In [ ]:
# 로지스틱 회귀분석 모델 생성
lr_clf = LogisticRegression() 

# 모델 학습 진행
lr_clf.fit(train_X, np.ravel(train_y))

# 모델 정확도 확인
train_y_pred = lr_clf.predict(train_X)
test_y_pred = lr_clf.predict(test_X)

print_score(train_y, train_y_pred, train = True)
print_score(test_y, test_y_pred, train = False)

# confusion matrix, AUC-ROC 커브 생성
disp = plot_confusion_matrix(lr_clf, test_X, test_y, cmap='Blues', values_format='d', display_labels=['Good', 'Bad'])
disp = plot_roc_curve(lr_clf, test_X, test_y)

(2) Random Forest 진행 (약 3분 30초 소요)

- Data Set에 대한 예측 정확도, F1 Score, AUC 값 모두 높다
- 역시 과적합 문제 발생

In [ ]:
# Random Forest 모델 생성
rf_clf = RandomForestClassifier(n_estimators = 50)    # n_estimators는 생성할 Tree의 개수. 개수가 많을수록 정확도가 올라가지만 모델 학습 시간이 길어짐

# 모델 학습 진행
rf_clf.fit(train_X, train_y)

# 모델 정확도 확인
train_y_pred = rf_clf.predict(train_X)
test_y_pred = rf_clf.predict(test_X)

print_score(train_y, train_y_pred, train = True)
print_score(test_y, test_y_pred, train = False)

# confusion matrix, AUC-ROC 커브 생성
disp = plot_confusion_matrix(rf_clf, test_X, test_y, cmap='Blues', values_format='d', display_labels=['Good', 'Bad'])
disp = plot_roc_curve(rf_clf, test_X, test_y)

(3) XGBoost 진행 (약 2분 30초 소요)

- 역시 결과는 좋다.. 과적합 해결이 필요하다~

In [ ]:
# XGBoost 모델 생성
xgb_clf = XGBClassifier(use_label_encoder = False)    # XGBoost 라이브러리 문서까지 찾아봤는데, 별다른 설명 없이 False를 추천한다 하여 False로 설정

# 모델 학습 진행
xgb_clf.fit(train_X, train_y)

# 모델 정확도 확인
train_y_pred = xgb_clf.predict(train_X)
test_y_pred = xgb_clf.predict(test_X)

print_score(train_y, train_y_pred, train = True)
print_score(test_y, test_y_pred, train = False)

# confusion matrix, AUC-ROC 커브 생성
disp = plot_confusion_matrix(xgb_clf, test_X, test_y, cmap='Blues', values_format='d', display_labels=['Good', 'Bad'])
disp = plot_roc_curve(xgb_clf, test_X, test_y)

(4) 3 모델의 AUC값을 비교한 AUC_ROC Curve

  - 모든 모델들이 좋은 AUC 값을 보인다... -> 과적합 문제 판단됨
  - 미세한 차이지만, XGBoost > Random Forest > Logistic Regression 순의 정확도를 보인다. (물론 이는 차후 다른 데이터 Set을 이용한 결과도 봐야 할 것)

In [ ]:
disp = plot_roc_curve(lr_clf, test_X, test_y)
plot_roc_curve(rf_clf, test_X, test_y, ax=disp.ax_)
plot_roc_curve(xgb_clf, test_X, test_y, ax=disp.ax_)

### 2-4. 임시 (넣고 싶은 변수 아무거나)

- 모든 존재하는 변수 목록은 아래과 같음 (기존의 것 + 생성한 것)
       'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'int_rate', 'installment', 'annual_inc', 'dti', 'delinq_2yrs', 'fico_range_low', 'fico_range_high', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_amnt', 'last_fico_range_high', 'last_fico_range_low', 'collections_12_mths_ex_med', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'chargeoff_within_12_mths', 'delinq_amnt', 'pub_rec_bankruptcies', 'tax_liens', 'emp_length1', 'emp_length2', 'emp_length3', 'emp_length4', 'emp_length5', 'emp_length6', 'emp_length7', 'emp_length8', 'emp_length9', 'emp_length10', 'emp_length11', 'emp_length12', 'home_ownership1', 'home_ownership2', 'home_ownership3', 'home_ownership4', 'home_ownership5', 'home_ownership6', 'verification_status1', 'verification_status2', 'verification_status3', 'purpose1', 'purpose2', 'purpose3', 'purpose4', 'purpose5', 'purpose6', 'purpose7', 'purpose8', 'purpose9', 'purpose10', 'purpose11', 'purpose12', 'purpose13', 'purpose14', 'addr_state1', 'addr_state2', 'addr_state3', 'addr_state4', 'addr_state5', 'addr_state6', 'addr_state7', 'addr_state8', 'addr_state9', 'addr_state10', 'addr_state11', 'addr_state12', 'addr_state13', 'addr_state14', 'addr_state15', 'addr_state16', 'addr_state17', 'addr_state18', 'addr_state19', 'addr_state20', 'addr_state21', 'addr_state22', 'addr_state23', 'addr_state24', 'addr_state25', 'addr_state26', 'addr_state27', 'addr_state28', 'addr_state29', 'addr_state30', 'addr_state31', 'addr_state32', 'addr_state33', 'addr_state34', 'addr_state35', 'addr_state36', 'addr_state37', 'addr_state38', 'addr_state39', 'addr_state40', 'addr_state41', 'addr_state42', 'addr_state43', 'addr_state44', 'addr_state45', 'addr_state46', 'addr_state47', 'addr_state48', 'addr_state49', 'addr_state50', 'addr_state51', 'initial_list_status1', 'initial_list_status2', 'elapsed_t', 'issue_d1', 'issue_d2', 'issue_d3', 'issue_d4', 'issue_d5', 'issue_d6', 'issue_d7', 'issue_d8', 'issue_d9', 'issue_d10', 'issue_d11', 'issue_d12', 'issue_d13', 'issue_d14', 'issue_d15', 'issue_d16', 'issue_d17', 'issue_d18', 'issue_d19', 'issue_d20', 'issue_d21', 'issue_d22', 'issue_d23', 'issue_d24', 'issue_d25', 'issue_d26', 'issue_d27', 'issue_d28', 'issue_d29', 'issue_d30', 'issue_d31', 'issue_d32', 'issue_d33', 'issue_d34', 'issue_d35', 'issue_d36', 'issue_d37', 'issue_d38', 'issue_d39', 'issue_d40', 'issue_d41', 'issue_d42', 'issue_d43', 'issue_d44', 'issue_d45', 'issue_d46', 'issue_d47', 'issue_d48', 'issue_d49', 'issue_d50', 'issue_d51', 'issue_d52', 'issue_d53', 'issue_d54', 'issue_d55', 'issue_d56', 'issue_d57', 'issue_d58', 'issue_d59', 'issue_d60', 'issue_d61', 'issue_d62', 'issue_d63', 'issue_d64', 'issue_d65', 'issue_d66', 'issue_d67', 'issue_d68', 'issue_d69', 'issue_d70', 'issue_d71', 'issue_d72', 'issue_d73', 'issue_d74', 'issue_d75', 'issue_d76', 'issue_d77', 'issue_d78', 'issue_d79', 'issue_d80', 'issue_d81', 'issue_d82', 'issue_d83', 'issue_d84', 'issue_d85', 'issue_d86', 'issue_d87', 'issue_d88', 'issue_d89', 'issue_d90', 'issue_d91', 'issue_d92', 'issue_d93', 'issue_d94', 'issue_d95', 'issue_d96', 'issue_d97', 'issue_d98', 'issue_d99', 'issue_d100', 'issue_d101', 'issue_d102', 'issue_d103', 'issue_d104', 'issue_d105', 'issue_d106', 'issue_d107', 'issue_d108', 'issue_d109', 'issue_d110', 'issue_d111', 'issue_d112', 'issue_d113', 'issue_d114', 'issue_d115', 'issue_d116', 'issue_d117', 'issue_d118', 'debt_settlement_flag1', 'term1', 'depvar', 'mths_since_last_delinq1', 'mths_since_last_delinq2', 'mths_since_last_delinq3', 'mths_since_last_delinq4', 'mths_since_last_delinq5', 'mths_since_last_delinq6', 'mths_since_last_delinq7', 'mths_since_last_delinq8', 'mths_since_last_delinq9', 'mths_since_last_delinq10', 'mths_since_last_delinq11', 'mths_since_last_major_derog1', 'mths_since_last_major_derog2', 'mths_since_last_major_derog3', 'mths_since_last_major_derog4', 'mths_since_last_major_derog5', 'mths_since_last_major_derog6', 'mths_since_last_major_derog7', 'mths_since_last_major_derog8', 'mths_since_last_major_derog9', 'mths_since_last_major_derog10', 'mths_since_last_major_derog11', 'mths_since_last_record1', 'mths_since_last_record2', 'mths_since_last_record3', 'mths_since_last_record4', 'mths_since_last_record5', 'mths_since_last_record6', 'mths_since_last_record7', 'mths_since_last_record8', 'mths_since_last_record9', 'mths_since_last_record10', 'mths_since_last_record11', 'mths_since_rcnt_il1', 'mths_since_rcnt_il2', 'mths_since_rcnt_il3', 'mths_since_rcnt_il4', 'mths_since_rcnt_il5', 'mths_since_rcnt_il6', 'mths_since_rcnt_il7', 'mths_since_rcnt_il8', 'mths_since_rcnt_il9', 'mths_since_rcnt_il10', 'mths_since_rcnt_il11', 'mths_since_recent_bc1', 'mths_since_recent_bc2', 'mths_since_recent_bc3', 'mths_since_recent_bc4', 'mths_since_recent_bc5', 'mths_since_recent_bc6', 'mths_since_recent_bc7', 'mths_since_recent_bc8', 'mths_since_recent_bc9', 'mths_since_recent_bc10', 'mths_since_recent_bc11', 'mths_since_recent_bc_dlq1', 'mths_since_recent_bc_dlq2', 'mths_since_recent_bc_dlq3', 'mths_since_recent_bc_dlq4', 'mths_since_recent_bc_dlq5', 'mths_since_recent_bc_dlq6', 'mths_since_recent_bc_dlq7', 'mths_since_recent_bc_dlq8', 'mths_since_recent_bc_dlq9', 'mths_since_recent_bc_dlq10', 'mths_since_recent_bc_dlq11', 'mths_since_recent_inq1', 'mths_since_recent_inq2', 'mths_since_recent_inq3', 'mths_since_recent_inq4', 'mths_since_recent_inq5', 'mths_since_recent_inq6', 'mths_since_recent_inq7', 'mths_since_recent_inq8', 'mths_since_recent_inq9', 'mths_since_recent_inq10', 'mths_since_recent_revol_delinq1', 'mths_since_recent_revol_delinq2', 'mths_since_recent_revol_delinq3', 'mths_since_recent_revol_delinq4', 'mths_since_recent_revol_delinq5', 'mths_since_recent_revol_delinq6', 'mths_since_recent_revol_delinq7', 'mths_since_recent_revol_delinq8', 'mths_since_recent_revol_delinq9', 'mths_since_recent_revol_delinq10', 'mths_since_recent_revol_delinq11', 'emp_length', 'home_ownership12', 'mths_since_rcnt_il_a', 'mths_since_rcnt_il_b', 'mths_since_rcnt_il_c', 'mths_since_recent_bc', 'mths_since_recent_inq_a', 'mths_since_recent_inq_b','mths_since_recent_inq_c', 'delinq_2yrs_a', 'delinq_2yrs_b', 'delinq_2yrs_c', 'fico_range_low_a', 'inq_last_6mths_a', 'inq_last_6mths_b', 'inq_last_6mths_c', 'collections_12_mths_ex_med_a', 'acc_now_delinq_a', 'pub_rec_bankruptcies_a', 'tax_liens_a', 'installment_a', 'annual_inc_a', 'collection_recovery_fee_a', 'dti_a'

- 더블클릭하면 한 페이지에 확인 가능
- 이들 중 필요하다고 생각하는 변수들만 뽑아 모델 생성 진행 (단, 종속변수 
'depvar'은 무조건 포함되어야 함)

- 변수 추출

In [ ]:
data = lc_df[['loan_amnt', 'int_rate', 'installment', 'annual_inc', 'dti', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 
              'pub_rec_bankruptcies', 'fico_range_low', 'last_fico_range_low', 'term1', 'home_ownership12', 'initial_list_status1', 'term1', 'emp_length12', 'depvar']]

In [ ]:
feature_columns = list(data.columns.difference(['depvar']))
X = data[feature_columns]    # 변인변수들의 데이터 Set
y = data['depvar']    # 종속변수 데이터

- 데이터 스케일링 진행

In [ ]:
# Robust Scaling 진행
rob = RobustScaler()
X = rob.fit_transform(X)

# 데이터형을 float32로 통일
train_X = np.array(train_X).astype(np.float32)
test_X = np.array(test_X).astype(np.float32)
train_y = np.array(train_y).astype(np.float32)
test_y = np.array(test_y).astype(np.float32)

- Train Set / Test Set 분류

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.2, random_state = 42)

print(train_X.shape, test_X.shape, train_y.shape, test_y.shape)

(1) 로지스틱 회귀 진행

In [ ]:
# 로지스틱 회귀분석 모델 생성
lr_clf = LogisticRegression() 

# 모델 학습 진행
lr_clf.fit(train_X, np.ravel(train_y))

# 모델 정확도 확인
train_y_pred = lr_clf.predict(train_X)
test_y_pred = lr_clf.predict(test_X)

print_score(train_y, train_y_pred, train = True)
print_score(test_y, test_y_pred, train = False)

# confusion matrix, AUC-ROC 커브 생성
# disp = plot_confusion_matrix(lr_clf, test_X, test_y, cmap='Blues', values_format='d', display_labels=['Good', 'Bad'])
# disp = plot_roc_curve(lr_clf, test_X, test_y)

(2) Random Forest 진행

In [ ]:
# Random Forest 모델 생성
rf_clf = RandomForestClassifier(n_estimators = 50)    # n_estimators는 생성할 Tree의 개수. 개수가 많을수록 정확도가 올라가지만 모델 학습 시간이 길어짐

# 모델 학습 진행
rf_clf.fit(train_X, train_y)

# 모델 정확도 확인
train_y_pred = rf_clf.predict(train_X)
test_y_pred = rf_clf.predict(test_X)

print_score(train_y, train_y_pred, train = True)
print_score(test_y, test_y_pred, train = False)

# confusion matrix, AUC-ROC 커브 생성
# disp = plot_confusion_matrix(rf_clf, test_X, test_y, cmap='Blues', values_format='d', display_labels=['Good', 'Bad'])
  # disp = plot_roc_curve(rf_clf, test_X, test_y)

(3) XGBoost 진행

In [ ]:
# XGBoost 모델 생성
xgb_clf = XGBClassifier(use_label_encoder = False)    # XGBoost 라이브러리 문서까지 찾아봤는데, 별다른 설명 없이 False를 추천한다 하여 False로 설정

# 모델 학습 진행
xgb_clf.fit(train_X, train_y)

# 모델 정확도 확인
train_y_pred = xgb_clf.predict(train_X)
test_y_pred = xgb_clf.predict(test_X)

print_score(train_y, train_y_pred, train = True)
print_score(test_y, test_y_pred, train = False)

# confusion matrix, AUC-ROC 커브 생성
# disp = plot_confusion_matrix(xgb_clf, test_X, test_y, cmap='Blues', values_format='d', display_labels=['Good', 'Bad'])
# disp = plot_roc_curve(xgb_clf, test_X, test_y)

(4) AUC 값 비교

In [ ]:
disp = plot_roc_curve(lr_clf, test_X, test_y)
plot_roc_curve(rf_clf, test_X, test_y, ax=disp.ax_)
plot_roc_curve(xgb_clf, test_X, test_y, ax=disp.ax_)

<참고 사이트> : 결과 해석에 도움이 될 것 같아서..

- Random Forest, XGBoost : https://www.kaggle.com/faressayah/lending-club-loan-defaulters-prediction
- 로지스틱 회귀 : https://datascienceschool.net/03%20machine%20learning/10.01%20%EB%A1%9C%EC%A7%80%EC%8A%A4%ED%8B%B1%20%ED%9A%8C%EA%B7%80%EB%B6%84%EC%84%9D.html
- AUC_ROC 곡선 : https://bioinformaticsandme.tistory.com/328
- Confusion Matrix : https://shinminyong.tistory.com/28

## 3. 변수 선택 : Lasso(L1), Ridge(L2) 적용
(1) 변수 추출 및 데이터 생성

(2) Train / Test Set 구분 (필요시 Scaling 진행)

(3) Logistic / Random Forest / XGBoost 모델 생성 및 정확도 평가 (필요시 다운샘플링 데이터 Set도 적용)


In [ ]:
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge